# Formatting Functions

In [ ]:
import scipy.io as sio
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from skimage.feature import hog
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import os
import scipy.sparse as sp
import scipy.io as sio
import time
import cuml
from cuml.svm import SVC as cuSVC
import cudf
from sklearn.decomposition import PCA
import scipy.stats
import librosa
from cuml.neighbors import KNeighborsClassifier as cuKNN
import cupy as cp
import h5py
from sklearn.preprocessing import StandardScaler
import gc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout, Activation, ReLU, Input, LeakyReLU
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.utils import class_weight
from scipy.stats import mode
import string

# to split data like papers for reproducibility
def split_trials(trial_data, num_gestures, training_trials, test_trials):

    # calculate number of trials from arrays given. Create 3D array to store indices for each trial, for each gesture
    num_trials = len(training_trials) + len(test_trials)
    trial_index = np.zeros((num_trials, num_gestures, 2))

    curr_label = 0                                     # label indexed from 0, so gesture 1 is label 0 etc.
    for i in range(1, len(trial_data)):
        curr_trial = trial_data[i]- 1              # minus 1 to make zero indexed - trial 1 is 0 etc.
        prev_trial = trial_data[i-1] - 1

        if curr_trial != prev_trial:                   # store index at beginning and end of trial

            # zero indexed
            if curr_trial == 0:                                 # if trial is zero (trial 1), that means it is a new gesture (label)
                trial_index[prev_trial][curr_label][1] = i      # store index for beginning of that trial, for that gesture
                curr_label += 1                                 # update gesture (label) by 1

                trial_index[curr_trial][curr_label][0] = i      # store index for end of that trial - it is an index for range() function, so this value will not be included, e.g. 2017, means up to 2016

            else:
                trial_index[prev_trial][curr_label][1] = i      # same as before, except there is no need to update gesture (label), as trial is between 1-6 (or 0-5 zero-indexed)
                trial_index[curr_trial][curr_label][0] = i

    # update last trial, last gesture (label) beginning index, as it is wrong!!
    trial_index[(num_trials-1)][(num_gestures-1)][0] = trial_index[(num_trials-2)][(num_gestures-1)][1]

    return trial_index

def split_labels(index_info, label_data, training_trials, test_trials):

    training_labels, test_labels = [], []

    # use the indices found to create arrays for training data (labels and features)
    for j in training_trials:
        trial_num = j-1

        for indices in index_info[trial_num]:
                beg, end = int(indices[0]), int(indices[1])

                # slice the training labels and features
                training_labels = np.append(training_labels, label_data[beg:end])

    # use the indices found to slice testing features and labels
    for j in test_trials:
        trial_num = j-1

        for indices in index_info[trial_num]:

                beg, end = int(indices[0]), int(indices[1])
                test_labels = np.append(test_labels, label_data[beg:end])

    return training_labels, test_labels

def split_time_series(index_info, time_series_arr, electrode_num, train_len, test_len, training_trials, test_trials):

    train_split_series = np.zeros((electrode_num, train_len))
    test_split_series = np.zeros((electrode_num, test_len))

    for num in range(electrode_num):

      training_series, test_series= [], []

      # use the indices found to create arrays for training data
      for j in training_trials:
          trial_num = j-1

          for indices in index_info[trial_num]:
              beg, end = int(indices[0]), int(indices[1])

              # slice the training labels and features
              training_series = np.append(training_series, time_series_arr[num, beg:end])

      train_split_series[num] = training_series

      # use the indices found to slice testing features and labels
      for i in test_trials:
          trial_num = i-1

          for indices in index_info[trial_num]:

              beg, end = int(indices[0]), int(indices[1])
              test_series = np.append(test_series, time_series_arr[num, beg:end])


      test_split_series[num] = test_series

    return train_split_series, test_split_series

def rolling_window(arr, window_len, step, arr_dimension):

    if arr_dimension == 2:
        num_windows = (arr.shape[0] - window_len) // step + 1
        windows = np.zeros((num_windows, window_len, arr.shape[1]), dtype=arr.dtype)
    elif arr_dimension == 1:
        num_windows = (len(arr) - window_len) // step + 1
        windows = np.zeros((num_windows, window_len), dtype=arr.dtype)

    for i in range(num_windows):
        start = i * step
        end = start + window_len
        windows[i] = arr[start:end]

    return windows

def rolling_window_electrodes(arr, window_len, step):

    num_windows = (arr.shape[1] - window_len) // step + 1
    windows = np.zeros((arr.shape[0], num_windows, window_len), dtype=arr.dtype)

    for i in range(num_windows):
      start = i * step
      end = start + window_len
      windows[:, i] = arr[:, start:end]  # Slice along columns

    return windows

def one_hot_encoder(labels, gestures):

        one_hot = np.zeros((len(labels), gestures))

        for index, value in enumerate(labels):
            label_encode = int(value)
            one_hot[index][label_encode] = 1

        return one_hot

class LocallyConnectedLayer(layers.Layer):
    def __init__(self, filters, kernel_size, **kwargs):
        super(LocallyConnectedLayer, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size

    def build(self, input_shape):
        # Initialize the weights for each spatial location
        self.kernel = self.add_weight(
            name='kernel',
            shape=(self.kernel_size[0], self.kernel_size[1], input_shape[-1], self.filters),
            initializer='glorot_uniform',
            trainable=True
        )
        super(LocallyConnectedLayer, self).build(input_shape)

    def call(self, inputs):
        # Perform the "local" convolution manually without weight sharing
        return tf.nn.conv2d(inputs, self.kernel, strides=[1, 1, 1, 1], padding='SAME')

def lr_schedule(epoch):
    initial_lr = 0.002  # Initial learning rate
    drop_factor = 0.5  # Factor by which to reduce the learning rate
    epochs_drop = 20  # Interval (in epochs) to reduce the learning rate
    return initial_lr * (drop_factor ** (epoch // epochs_drop))

def generate_signal_permutations(Ns):

    base = 36
    SIS = ""  # String to store signal indices
    NSIS = 1  # Initial length of SIS

    # Start with the first signal sequence (hexadecimal '1')
    #SIS = format(1, 'X')  # SIS starts with '1' in hexadecimal
    SIS = int_to_base(1, base)


    i = 1
    j = i + 1

    # Apply the permutation logic as per the original algorithm
    while i != j:
        if j > Ns:
            j = 1  # Wrap around if j exceeds Ns
        #elif format(i, 'X') + format(j, 'X') not in SIS and format(j, 'X') + format(i, 'X') not in SIS:  # If pair not in SIS
        elif int_to_base(i, base) + int_to_base(j, base) not in SIS and int_to_base(j, base) + int_to_base(i, base) not in SIS:
            # Append the j-th signal index to SIS (in hexadecimal)
            #SIS += format(j, 'X')  # Add j to the string in hexadecimal
            SIS += int_to_base(j, base)
            NSIS += 1  # Increment NSIS
            i = j  # Update i to current j
            j = i + 1  # Update j to i + 1
        else:
            j += 1  # Move j to the next signal sequence
    # Return the final Signal Index String (SIS) as a list of integers and NSIS
    #return [int(x, 16) for x in SIS], NSIS
    return [int(x, base) for x in SIS], NSIS

def int_to_base(n, base):

    digits = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    # Create a string with all digits and letters for bases up to 36
    digits = string.digits + string.ascii_lowercase
    if n == 0:
        return digits[0]

    result = []
    while n:
        result.append(digits[n % base])
        n //= base
    return ''.join(reversed(result))


# TO RUN

In [ ]:
import numpy as np
from google.colab import drive
# ADJUST ACCORDINLY
database = 'DB4'

# ADJUST ACCORDINLY
evaluation = 'HHT'

# ADJUST DEPNDING ON WHAT FEATURES
num_features = 3

# ADJUST DEPNDING ON Signal Image or not
signal_image = True

# ok, so E2 and E3 adjusted means to adjust them gesture back to 1-N gestures, otherwise it is say 18-40 but depending on what gesture set went first
data_dict = {
        'DB1': {'E1': 13, 'E2': 18, 'E3': 24, 'E1_adjusted': 0, 'E2_adjusted': 0, 'E3_adjusted': 0, 'fs': 100, 'electrodes': 10, 'subjects': 27, 'train': [1, 3, 4, 6, 7, 8, 9], 'test': [2, 5, 10], 'window length': 20, 'step': 1},
        'DB2': {'E1': 18, 'E2': 24, 'E3': 10, 'E1_adjusted': 0, 'E2_adjusted': -17, 'E3_adjusted': -40, 'fs': 100, 'electrodes': 12, 'subjects': 40, 'train': [1, 3, 4, 6], 'test': [2, 5], 'window length': 20, 'step': 1},
        'DB3': {'E1': 18, 'E2': 24, 'E3': 10, 'E1_adjusted': 0, 'E2_adjusted': -17, 'E3_adjusted': -40, 'fs': 200, 'electrodes': 12, 'subjects': 11, 'train': [1, 3, 4, 6], 'test': [2, 5], 'window length': 40, 'step': 2},
        'DB4': {'E1': 13, 'E2': 18, 'E3': 24, 'E1_adjusted': 0, 'E2_adjusted': 0, 'E3_adjusted': 0,'fs': 200, 'electrodes': 12, 'subjects': 10, 'train': [1, 3, 4, 6], 'test': [2, 5], 'window length': 40, 'step': 2},
        'DB5': {'E1': 13, 'E2': 18, 'E3': 24, 'E1_adjusted': 0, 'E2_adjusted': 0, 'E3_adjusted': 0,'fs': 200, 'electrodes': 16, 'subjects': 10, 'train': [1, 3, 4, 6], 'test': [2, 5], 'window length': 40, 'step': 2}
        }

num_subjects = data_dict[database]['subjects']
fs = data_dict[database]['fs']
num_electrodes = data_dict[database]['electrodes']

train_trials =  data_dict[database]['train']
test_trials = data_dict[database]['test']
M, step = data_dict[database]['window length'], data_dict[database]['step']
num_freq_bins = int((fs / 2) / (1 / (1/fs * M)))
freq_bins = np.linspace(0, fs/2, num_freq_bins)


drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# simulate button click to prevent server going idle
%%javascript
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

# Training

In [ ]:

for exercise in ['E1', 'E2', 'E3']:

  label_dict = {f'S{num}': [] for num in range(1, num_subjects+1)}
  features_dict = {f'S{num}': [] for num in range(1, num_subjects+1)}
  num_gestures = data_dict[database][exercise]

  # for all subjects
  for subject in range(1,(num_subjects+1)):


      # subjects 6 and 7 were not evaulated for DB3
      if database == 'DB3':
        if subject in [6,7]:
          continue

      # load labels and features for subject - DB2, DB3, DB5
      file = sio.loadmat(f'/content/drive/My Drive/uni/{database}/Electrode Data/S{subject}_{exercise}_A1.mat')

      label = file['restimulus'].flatten()
      trials = np.int8(file['rerepetition']).flatten()

      if database in ['DB3', 'DB4']:
        downsample_factor = 10
        label = label[::downsample_factor]
        trials = trials[::downsample_factor]

      # find indexes of where trials begin and end
      trial_split_index = split_trials(trials, num_gestures, train_trials, test_trials)
      # split labels with trial info
      train_labels, test_labels = split_labels(trial_split_index, label, train_trials, test_trials)

      # load train features
      train_feature_path = f'/content/drive/My Drive/uni/{database}/Features_down/Training_{exercise}_S{subject}_features.h5'
      with h5py.File(train_feature_path, 'r') as train_file:
          train_mav = train_file['MAV'][:]
          train_mavs = train_file['MAVS'][:]
          train_wap = train_file['WAP'][:]
          train_zcr = train_file['ZC'][:]
          train_ssc = train_file['SSC'][:]
          train_ar1 = train_file['ar1'][:]
          train_ar2 = train_file['ar2'][:]
          train_ar3 = train_file['ar3'][:]
          train_ar4 = train_file['ar4'][:]
          train_wl = train_file['WL'][:]
          train_rms = train_file['RMS'][:]
          train_ssc = train_file['SSC'][:]
          train_var = train_file['VAR'][:]
          train_iemg = train_file['IEMG'][:]

      # load either HHT or STFT data
      if evaluation == 'HHT':
          hht_test_path = f'/content/drive/My Drive/uni/{database}/HHT/Test_{exercise}_S{subject}_hht.h5'
          hht_train_path = f'/content/drive/My Drive/uni/{database}/HHT/Training_{exercise}_S{subject}_hht.h5'
      elif evaluation == 'STFT':
          hht_test_path = f'/content/drive/My Drive/uni/{database}/STFT/Test_{exercise}_S{subject}_stft.h5'
          hht_train_path = f'/content/drive/My Drive/uni/{database}/STFT/Training_{exercise}_S{subject}_stft.h5'

      with h5py.File(hht_train_path, 'r') as hht_file:
          train_mean_freq = hht_file['mean freq'][:]
          train_skew_freq = hht_file['skew freq'][:]
          train_psr = hht_file['psr'][:]
          #train_imfs = hht_file['num imfs'][:]
          train_peak_freq = hht_file['peak freq'][:]
          train_mean_power = hht_file['mean power'][:]
          train_kurt_freq = hht_file['kurt freq'][:]
          train_var_freq = hht_file['var freq'][:]

      with h5py.File(hht_test_path, 'r') as hht_file:
          test_mean_freq = hht_file['mean freq'][:]
          test_skew_freq = hht_file['skew freq'][:]
          test_psr = hht_file['psr'][:]
          #test_imfs = hht_file['num imfs'][:]
          test_peak_freq = hht_file['peak freq'][:]
          test_mean_power = hht_file['mean power'][:]
          test_kurt_freq = hht_file['kurt freq'][:]
          test_var_freq = hht_file['var freq'][:]

      if evaluation == 'HHT':
          train_mean_power = np.squeeze(train_mean_power)
          train_mean_freq = np.squeeze(train_mean_freq)
          train_psr = np.squeeze(train_psr)
          test_mean_power = np.squeeze(test_mean_power)
          test_mean_freq = np.squeeze(test_mean_freq)
          test_psr = np.squeeze(test_psr)

      # FEATURE SET 1 - 11 features
      #train_matrices = [train_mav, train_mavs, train_wap, train_zcr, train_ar1, train_ar2, train_ar3, train_ar4, train_wl, train_mean_freq, train_psr]

      # FEATURE SET 2 - 6 features
      #train_matrices = [train_iemg, train_var, train_wap, train_wl, train_ssc, train_zcr]

      # FEATURE SET 3 - 3 features
      train_matrices = [train_mean_power, train_wl, train_mav]

      # reshape array
      train_images = np.stack(train_matrices, axis=-1).transpose(1, 0, 2)

      # load test features
      test_feature_path = f'/content/drive/My Drive/uni/{database}/Features_down/Test_{exercise}_S{subject}_features.h5'
      with h5py.File(test_feature_path, 'r') as test_file:
          test_mav = test_file['MAV'][:]
          test_mavs = test_file['MAVS'][:]
          test_wap = test_file['WAP'][:]
          test_zcr = test_file['ZC'][:]
          test_ssc = test_file['SSC'][:]
          test_ar1 = test_file['ar1'][:]
          test_ar2 = test_file['ar2'][:]
          test_ar3 = test_file['ar3'][:]
          test_ar4 = test_file['ar4'][:]
          test_wl = test_file['WL'][:]
          test_rms = test_file['RMS'][:]
          test_ssc = test_file['SSC'][:]
          test_var = test_file['VAR'][:]
          test_iemg = test_file['IEMG'][:]

      # FEATURE SET 1 - 11 features
      #test_matrices = [test_iemg, test_var, test_wap, test_wl, test_ssc, test_zcr]

      # FEATURE SET 2 - 6 features
      #test_matrices = [test_mav, test_mavs, test_wap, test_zcr, test_ar1, test_ar2, test_ar3, test_ar4, test_wl, test_mean_freq, test_psr]

      # FEATURE Set 3 - 3 features
      test_matrices = [test_mean_power, test_wl, test_mav]

      # reshape array
      test_images = np.stack(test_matrices, axis=-1).transpose(1, 0, 2)

      # if testing different permutations of rows
      if signal_image == True:

        # use algorithm to get row order
        Ns = train_images.shape[1]
        SIS, NSIS = generate_signal_permutations(Ns)
        print(SIS, NSIS)

        # reshape train features to signal image
        train_features = np.zeros((train_images.shape[0], NSIS, train_images.shape[2]))
        print(train_features.shape)
        for segment, image in enumerate(train_images):
          for index, val in enumerate(SIS):
            train_features[segment][index] = train_images[segment, val-1, :]

         # reshape test features to signal image
        print(train_features.shape)
        test_features = np.zeros((test_images.shape[0], NSIS, test_images.shape[2]))
        for segment, image in enumerate(test_images):
          for index, val in enumerate(SIS):
            test_features[segment][index] = test_images[segment, val-1, :]

        # remove last two rows
        test_features = test_features[:, :-2, :]
        train_features = train_features[:, :-2, :]

      else:
        train_features = train_images
        test_features = test_images

      # window labels - only to predict whether gesture or not
      test_label_arr = rolling_window(test_labels, M, step, 1)
      train_label_arr = rolling_window(train_labels, M, step, 1)

      # format labels
      train_label_one = [np.max(arr) for arr in train_label_arr]
      test_label_one = [np.max(arr) for arr in test_label_arr]

      print(np.unique(train_label_one))
      print(np.unique(test_label_one))

      # save gesture before bringing index back to 1, as it will only be compared - save train / test features to be reloaded for validation
      with h5py.File(f'/content/drive/MyDrive/uni/{database}/History_unified/{exercise}_label_history_S{subject}.h5', 'w') as f:
        f.create_dataset(f'{exercise}_S{subject}', data=test_label_one, compression='gzip', compression_opts=8)

      with h5py.File(f'/content/drive/MyDrive/uni/{database}/History_unified/{exercise}_features_history_S{subject}.h5', 'w') as f:
          f.create_dataset(f'{exercise}_S{subject}', data=test_features, compression='gzip', compression_opts=8)

      # adjust gestures back to 1-N - it's a list
      train_label_ = [x + data_dict[database][f'{exercise}_adjusted'] if x != 0 else 0 for x in train_label_one]
      test_label_ = [x + data_dict[database][f'{exercise}_adjusted'] if x != 0 else 0 for x in test_label_one]
      print(np.unique(train_label_))
      print(np.unique(test_label_))
      train_label = one_hot_encoder(train_label_, num_gestures)
      test_label = one_hot_encoder(test_label_, num_gestures)

      print("fitting model now")

      print(train_features.shape, train_label.shape)
      print(test_features.shape, test_label.shape)

      # Define the model
      model = Sequential([

              Input(shape=(train_features.shape[1], num_features, 1)),
              BatchNormalization(),

              Conv2D(64, (3,3), padding='same', strides=(1,1)),
              BatchNormalization(),
              ReLU(),

              Conv2D(64, (3,3), padding='same', strides=(1,1)),
              BatchNormalization(),
              ReLU(),

              LocallyConnectedLayer(64, (1,1)),
              BatchNormalization(),
              ReLU(),

              LocallyConnectedLayer(64, (1,1)),
              BatchNormalization(),
              ReLU(),

              Dropout(0.5),

              Flatten(),
              Dense(512),
              BatchNormalization(),
              ReLU(),
              Dropout(0.65),
              Dense(512),
              BatchNormalization(),
              ReLU(),
              Dropout(0.65),

              # predict what gesture out of that exercise - even if subject didn't perform that gesture (for unknown reason) it could be predicted
              Dense(num_gestures, activation='softmax'),
      ])

      model.compile(optimizer=Adam(learning_rate=0.002), loss='categorical_crossentropy', metrics=['accuracy'])
      early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)

      checkpoint_weights = ModelCheckpoint(f'/content/drive/MyDrive/uni/{database}/Weights_unified/{exercise}_model_weights_S{subject}.weights.h5',
                                      save_best_only=True,
                                      save_weights_only=True,
                                      monitor='val_accuracy',)
                                      #verbose=1)

      epoch_num = 200
      history = model.fit(
                train_features,
                train_label,
                epochs=epoch_num,
                batch_size=32,
                validation_data=(test_features, test_label),
                callbacks=[checkpoint_weights, LearningRateScheduler(lr_schedule), early_stopping],
                )


# Validation

In [ ]:
from sklearn.metrics import accuracy_score
import seaborn as sns
import scipy.sparse as sp

tot_num_gestures = data_dict[database]['E1'] + data_dict[database]['E2'] + data_dict[database]['E3'] - 2

combined_cm = np.zeros((tot_num_gestures, tot_num_gestures), dtype=int)
accuracy_dict = {'E1': [], 'E2': [], 'E3': []}

Ns = data_dict[database]['electrodes']
SIS, NSIS = generate_signal_permutations(Ns)

for exercise in ['E1', 'E2', 'E3']:

  num_gestures = data_dict[database][exercise]

  # Define the model
  model = Sequential([

              # CHANGE THIS DEPENDING ON NUMBER OF ROWS IN FEATURES
              Input(shape=(NSIS-2, num_features, 1)),
              BatchNormalization(),

              Conv2D(64, (3,3), padding='same', strides=(1,1)),
              BatchNormalization(),
              ReLU(),

              Conv2D(64, (3,3), padding='same', strides=(1,1)),
              BatchNormalization(),
              ReLU(),

              LocallyConnectedLayer(64, (1,1)),
              BatchNormalization(),
              ReLU(),

              LocallyConnectedLayer(64, (1,1)),
              BatchNormalization(),
              ReLU(),

              Dropout(0.5),

              Flatten(),
              Dense(512),
              BatchNormalization(),
              ReLU(),
              Dropout(0.65),
              Dense(512),
              BatchNormalization(),
              ReLU(),
              Dropout(0.65),

              # predict what gesture out of that exercise - even if subject didn't perform that gesture (for unknown reason) it could be predicted
              Dense(num_gestures, activation='softmax'),
  ])

  for subject in range(1, num_subjects+1):

      # these subjects in DB3 are not used
      if database == 'DB3':
        if subject in [6,7]:
          continue

      # load saved test and train features
      with h5py.File(f'/content/drive/MyDrive/uni/{database}/History_unified/{exercise}_features_history_S{subject}.h5', 'r') as f:
        test_features = f[f'{exercise}_S{subject}'][:]

      with h5py.File(f'/content/drive/MyDrive/uni/{database}/History_unified/{exercise}_label_history_S{subject}.h5', 'r') as f:
        test_label = f[f'{exercise}_S{subject}'][:]


      # load labels and features for subject
      model.load_weights(f'/content/drive/MyDrive/uni/{database}/Weights_unified/{exercise}_model_weights_S{subject}.weights.h5')

      print(np.unique(test_label))
      # compile the model
      predictions = model.predict(test_features)
      predicted_classes = np.argmax(predictions, axis=1)

      # adjust predicted_classes back to corresponding gesture
      #predicted_classes = [(x - data_dict[database][f'{exercise}_adjusted']) for x in predicted_classes]
      predicted_classes = [x - data_dict[database][f'{exercise}_adjusted'] if x != 0 else x for x in predicted_classes]
      accuracy = accuracy_score(test_label, predicted_classes)
      print(f'{exercise} S{subject} Accuracy: {accuracy*100:.2f}%')

      # create confusion matrix
      for pred, actual in zip(predicted_classes, test_label):
        '''
        if pred != 0 and actual != 0:
          if exercise == 'E2':
            pred += 12
            actual += 12
          elif exercise == 'E3':
            pred += 29
            actual += 29
          '''
        combined_cm[int(actual), int(pred)] += 1

      accuracy_dict[exercise].append(accuracy*100)

for key in accuracy_dict.keys():
  print(f'Average accuracy for {key}: {np.mean(accuracy_dict[key])}, and std: {np.std(accuracy_dict[key])}')
  print(f'num elements {len(accuracy_dict[key])}')

ye = []
for val in accuracy_dict.values():
  ye.extend(val)
print(f'Overall average: {np.mean(ye)} and std: {np.std(ye)}')

#cm_path = f'/content/drive/MyDrive/uni/{database}/Confusion_matrices/{database}_confusion_matrix_phin_stft.npz'
#sp.save_npz(cm_path, sp.csr_matrix(combined_cm))

# adjust labels, because zero is not included and all exercises
display_labels = [str(i) for i in range(tot_num_gestures)]
plt.figure(figsize=(8, 6))
sns.heatmap(combined_cm, annot=False, cmap='inferno', fmt='g', cbar=True, square=True)
plt.xticks(ticks=np.arange(0, len(display_labels), 2), labels=[display_labels[i] for i in range(0, len(display_labels), 2)], rotation=45)
plt.yticks(ticks=np.arange(0, len(display_labels), 2), labels=[display_labels[i] for i in range(0, len(display_labels), 2)], rotation=45)

plt.xlabel('Predicted Labels', fontsize=12)
plt.ylabel('True Labels', fontsize=12)
plt.title(f"Classification Accuracy of {database} for {data_dict[database]['subjects']} Subjects")
plt.show()